In [149]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, root_mean_squared_error

# Carregar e visualizar os dados

In [150]:
# Carrear os dados - Regressão
df = pd.read_csv('datasets/dataset_colesterol.csv')

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [152]:
# Ajustar o df
df.drop('Id', axis=1, inplace=True)

# Coletar medidas das variaveis categoricas
moda_grupo_sanguineo = df['Grupo Sanguíneo'].mode()
moda_fumante = df['Fumante'].mode()
moda_nivel_atividade_fisica = df['Nível de Atividade'].mode()

# Coletar medidas das variaveis numericas
media_idade = df['Idade'].mean()
media_peso = df['Peso'].mean()   
media_altura = df['Altura'].mean()

# Tratar valores ausentes
df.fillna(value={
    'Grupo Sanguíneo': moda_grupo_sanguineo[0],
    'Fumante': moda_fumante[0],
    'Nível de Atividade': moda_nivel_atividade_fisica[0],
    'Idade': media_idade,
    'Peso': media_peso,
    'Altura': media_altura    
}, inplace=True)

# Aplicar One Hot Encoding nas variáveis categóricas
df = pd.get_dummies(df, columns=["Grupo Sanguíneo", "Fumante", "Nível de Atividade"])

In [153]:
# DF atualizado
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Idade                        1000 non-null   float64
 1   Peso                         1000 non-null   float64
 2   Altura                       1000 non-null   float64
 3   Colesterol                   1000 non-null   float64
 4   Grupo Sanguíneo_A            1000 non-null   uint8  
 5   Grupo Sanguíneo_AB           1000 non-null   uint8  
 6   Grupo Sanguíneo_B            1000 non-null   uint8  
 7   Grupo Sanguíneo_O            1000 non-null   uint8  
 8   Fumante_Não                  1000 non-null   uint8  
 9   Fumante_Sim                  1000 non-null   uint8  
 10  Nível de Atividade_Alto      1000 non-null   uint8  
 11  Nível de Atividade_Baixo     1000 non-null   uint8  
 12  Nível de Atividade_Moderado  1000 non-null   uint8  
dtypes: float64(4), uint

# Treinamento modelo de regressão linear multipla com RFE

In [154]:
# Separar X e Y
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [155]:
# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [156]:
# Treinar modelo com RFE - Recursive Feature Elimination
# RFE: Seleciona as melhores features para o modelo (Hiperparâmentros). Usando um estimador e define um número de features a serem selecionadas
# O RFE faz iterações iniciando com todas as features e eliminando uma a uma, até chegar no número de features desejado
# Eliminando as features menos importantes
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=6)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [157]:
# Quais features foram selecionadas
X_train.columns[rfe_method.get_support()]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Grupo Sanguíneo_B',
       'Fumante_Sim', 'Nível de Atividade_Alto'],
      dtype='object')

In [158]:
# Ranking de features
def mostrar_ranking(rfe_method, X_train):
    # Ranking de features
    ranking = rfe_method.ranking_

    # Obter nomes das features
    nomes_features = X_train.columns.tolist()

    # Crie um dataframe com os ranking e nomes das features
    df_ranking = pd.DataFrame({'Feature': nomes_features, 'Ranking': ranking})

    # Ordene o dataframe pelo ranking
    df_ranking = df_ranking.sort_values('Ranking')
    
    # Mostrar o ranking
    print(df_ranking)

In [159]:
# Ranking de features do RFE Regressão
mostrar_ranking(rfe_method, X_train)

                        Feature  Ranking
1                          Peso        1
2                        Altura        1
4            Grupo Sanguíneo_AB        1
5             Grupo Sanguíneo_B        1
8                   Fumante_Sim        1
9       Nível de Atividade_Alto        1
7                   Fumante_Não        2
3             Grupo Sanguíneo_A        3
11  Nível de Atividade_Moderado        4
10     Nível de Atividade_Baixo        5
6             Grupo Sanguíneo_O        6
0                         Idade        7


In [160]:
# Mostrar a perfornance do modelo
def mostrar_performance(modelo, X_test, y_test):
    # Fazer previsões
    y_pred = modelo.predict(X_test)

    # Calcular o erro
    return root_mean_squared_error(y_test, y_pred)

In [161]:
# Performance Regressão com RFE
mostrar_performance(rfe_method, X_test, y_test)

8.998690714652632

# Treinar modelo sem RFE

In [162]:
# Treinar Modelo de regressão sem RFE
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

LinearRegression()

In [163]:
# Performance do modelo de regressão sem RFE
mostrar_performance(model_reg, X_test, y_test)

9.042033118709876

# Treinar modelo de regressão linear com RFECV (Recursive Feature Elimination with Cross-Validation)

In [164]:
# Treinar modelo com RFECV - Recursive Feature Elimination Cross Validation
rfe_method_cv = RFECV(estimator=LinearRegression(), cv=5, min_features_to_select=6)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [165]:
mostrar_performance(rfe_method_cv, X_test, y_test)

9.049297523860693

In [166]:
# Quais features foram escolhidas?
X_train.columns[rfe_method_cv.get_support()]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [167]:
# Quantas features foram escolhidas?
rfe_method_cv.n_features_

10

# Treinar modelo de regressão com SelectFromModel

In [182]:
# Treinar modelo com SelectFromModel
sfm_method = SelectFromModel(estimator=model_reg, max_features=4, threshold=0.01)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=4, threshold=0.01)

In [183]:
# Quais features foram escolhidas?
X_train.columns[sfm_method.get_support()]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Nível de Atividade_Alto'], dtype='object')

In [184]:
# Treinar modelo com as features selecionadas
X_train_sfm = sfm_method.transform(X_train)
X_test_sfm = sfm_method.transform(X_test)
model_reg.fit(X_train_sfm, y_train)

LinearRegression()

In [185]:
# Performance do modelo com as features selecionadas
mostrar_performance(model_reg, X_test_sfm, y_test)

9.036994299376598